In [1]:
import pandas as pd
import numpy as np

data = 'D:\장우영\LOCALSEARCH\DA\DA\data\FAmerge_20230518_124958.csv'

merge_data = pd.read_csv(data, encoding='ANSI')
#print(weather_data)
print(merge_data.head(1))

        mmsi ship_name  ship_type   
0  440051540      D-01          0  \

                                                geom    cog  sog   
0  0101000020110F00000000002039676B41000000002DCF...  329.2  5.7  \

        insert_time  지방청           표지                일시   풍향   유향    기온    수온   
0  2023-05-11 10:10  부산청  신항유도등부표(랜비)  2023-05-11 10:10  196  246  16.4  17.0  \

     풍속   유속    기압  습도  
0  9.77  0.3  1017  82  
